In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

/opt/homebrew/Caskroom/miniforge/base/envs/FoundationalAI/lib/python3.10/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/opt/homebrew/Caskroom/miniforge/base/envs/FoundationalAI/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <0B7EB158-53DC-3403-8A49-22178CAB4612> /opt/homebrew/Caskroom/miniforge/base/envs/FoundationalAI/lib/python3.10/site-packages/torchvision/image.so
  Reason: tried: '/opt/homebrew/Caskroom/miniforge/base/envs/FoundationalAI/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/homebrew/Caskroom/miniforge/base/envs/FoundationalAI/lib/python3.10/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/opt/homebrew/Caskroom/miniforge/base/envs/FoundationalAI/lib/python3.10/lib-dynload/../../libjpeg.9.dylib' (no such file), '/opt/homebrew/Caskroom/miniforge/base/envs/FoundationalAI/bin/../lib/li

# Download and setup FashionMNIST Dataset

In [2]:
# this transform will convert data to tensor then standardize the data (precomputed mean and std)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.2860,), (0.3530,))])
# download training and testing and apply transform to both
training_data = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
testing_data = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

In [3]:
# these data loaders handle the data during training and testing. Increase testing batch size if you have a lot of RAM
trainloader = torch.utils.data.DataLoader(training_data, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testing_data, batch_size=1000, shuffle=False)

# Design Model

In [80]:
class MyCNN(nn.Module):
    """
    Create a custom CNN for FashionMNIST.

    The default is a really, awful design
    """
    def __init__(self):
        # initialize nn.Module
        super(MyCNN, self).__init__()
        # A layer of 3x3 conv with 32 filters, output will be same heigh/width as input (which are 28x28)
        # output will be of shape (batch size, 32, 28, 28)
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding='same')
        self.conv2 = nn.Conv2d(1, 16, kernel_size=5, padding='same')

        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, padding='same')
        self.conv4 = nn.Conv2d(32, 32, kernel_size=7, padding=0)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        # fully connected layer connecting output of pool to next dense layer
        self.fc1 = nn.Linear(32, 16)
        self.fc2 = nn.Linear(16, 10)
        # we will use ReLU for all activations (except output layer)
        self.relu = nn.ReLU()
        # self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        """
        Forward method for network
        :param x: input, for fashion MNIST, this is of shape (batch size, 28, 28)
        :return: output logits, shape (batch size, 10)
        """
        x1 = self.relu(self.conv1(x))
        # x1 = self.dropout(x1)
        x1 = self.pool(x1)

        x2 = self.relu(self.conv2(x))
        # x2 = self.dropout(x2)
        x2 = self.pool(x2)

        x = torch.cat([x1, x2], dim=1) # this will be 32x14x14
        # print(x.shape)

        x = self.relu(self.conv3(x))
        # x = self.dropout(x)
        x = self.pool(x)  # this will be 64x7x7
        # print(x.shape)

        x = self.relu(self.conv4(x))
        # print(x.shape)

        x = x.view(-1, 32)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [81]:
# Initialize model, loss, and optimizer
device = torch.device("mps" if torch.cuda.is_available() else "cpu")
model = MyCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [82]:
# Training function
def train_model(model, trainloader, criterion, optimizer, epochs=5):
    for epoch in range(epochs):
        running_loss = 0.0
        correct, total = 0, 0

        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}, Accuracy: {100 * correct / total:.2f}%")


In [83]:
# Train model
train_model(model, trainloader, criterion, optimizer, epochs=10)

Epoch 1, Loss: 0.5117, Accuracy: 81.15%
Epoch 2, Loss: 0.3201, Accuracy: 88.48%
Epoch 3, Loss: 0.2703, Accuracy: 90.24%
Epoch 4, Loss: 0.2446, Accuracy: 91.12%
Epoch 5, Loss: 0.2241, Accuracy: 91.82%
Epoch 6, Loss: 0.2076, Accuracy: 92.39%
Epoch 7, Loss: 0.1930, Accuracy: 93.03%
Epoch 8, Loss: 0.1799, Accuracy: 93.29%
Epoch 9, Loss: 0.1709, Accuracy: 93.69%
Epoch 10, Loss: 0.1606, Accuracy: 94.06%


In [84]:
# Evaluation function
def evaluate_model(model, testloader):
    model.eval()
    correct, total = 0, 0
    all_predictions = []

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.cpu().tolist())

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

    return all_predictions

In [85]:
# Evaluate model
predictions = evaluate_model(model, testloader)

Test Accuracy: 90.89%


In [86]:
# Save predictions for submission
import json
submission_data = {"name": "Student_Name", "predictions": predictions}
with open("submission.json", "w") as f:
    json.dump(submission_data, f)

In [87]:
# Save trained model
torch.save(model, "model.pth")


In [88]:
import requests
# Send results
data = {
    "name": "The Real Ghawaly :(",
    "predictions": predictions
}
response = requests.post("https://csc7700leaderboard-d4fce9d9h2b5h8ab.centralus-01.azurewebsites.net/submit", json=data)
print(response.json())

{'message': 'Submission received. Accuracy: 90.89%'}
